<a href="https://colab.research.google.com/github/951237/TIL/blob/master/%EC%9B%B9%EC%95%B1_%EB%82%A0%EC%94%A8%EC%98%88%EC%B8%A1_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8_200503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TIL - 200210 : 배추가격 예측 AI로봇 
- 링크 : https://youtu.be/wZvhBVqitn8

## TIL - 200209 텐서플로 따라하기(안경잡이 개발자)
- x축은 시간 
- y축은 임금
- 데이터를 기반으로 추후 임금 구하기

### 데이터 수집
- 기상자료개방포털 
    - 기온 : https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
    - 강수량 : https://data.kma.go.kr/stcs/grnd/grndRnList.do?pgmNo=69
- 농산물 유통정보 / 채소류 / 배추 / 전체품종
    - 소매가격 / 최소가격

### 데이터 정제
- 평균기온, 최저기온, 최고기온, 강수량, 가격

### 텐서플로 다변인 선형 모델 계발하기
- 변수가 여러가지 인경우 다변인 선형 회귀모델 적용
    - 식 : H(x1, x2, x3, x4) = x1w1 + x2w2 + x3w3 + x4w4 
        - 모든 변인이 가격에 영향을 미친다고 감안. 가중치를 고려하여 식을 세움.
        - 행렬의 곱을 이요한 식 : H(X) = XW
    - 학습률 : 0.000005

## 텐서플로 2.0 변환 성공

참고 : https://leejigun.github.io/tensorflow2_2

멀티회구 예측 : https://leejigun.github.io/tensorflow2_3

다변인 선형회귀(good) : https://sdc-james.gitbook.io/onebook/4.-and/5.4.-tensorflow/5.4.1.-keras

추가작업 : 학습중 결과 보여주기

### 훈련하기

In [10]:

# 라이브러리 호출
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import numpy as np
import pandas as pd

# 실행할때마다 같은 결과를 출력하기 위해 설정하는 부분
np.random.seed(3)
tf.random.set_seed(3)

# 데이터 불러오기
data = pd.read_csv('/content/drive/My Drive/data/ML/배추가격_데이터_예제_price data.csv')   # 데이터 파일 불러오기

# 학습데이터 준비하기
xy = np.array(data, dtype=np.float32)       # 데이터를 넘파이 값으로 담기 
x_data = xy[:, 1:-1]    # 모든행을 선택, 열은 2번째 부터 끝에서 2번째 열까지 선택(변수 4가지)
y_data = xy[:, [-1]]    # 모든행, 마지막 열 선택(가격)

# 모델 설정하기
model = Sequential()
model.add(Dense(1, input_dim = 4))
sgd = SGD(learning_rate=0.0005)

# 모델 컴파일
model.compile(loss='mean_squared_error', optimizer=sgd)
model.summary

model.fit(x_data, y_data, epochs = 1000)



Epoch 1/1000
92/92 [==============================] - 0s 1ms/step - loss: 3006059.0000
Epoch 2/1000
92/92 [==============================] - 0s 1ms/step - loss: 2345747.2500
Epoch 3/1000
92/92 [==============================] - 0s 1ms/step - loss: 2325028.5000
Epoch 4/1000
92/92 [==============================] - 0s 1ms/step - loss: 2344284.7500
Epoch 5/1000
92/92 [==============================] - 0s 1ms/step - loss: 2337895.5000
Epoch 6/1000
92/92 [==============================] - 0s 1ms/step - loss: 2303650.5000
Epoch 7/1000
92/92 [==============================] - 0s 1ms/step - loss: 2353222.0000
Epoch 8/1000
92/92 [==============================] - 0s 1ms/step - loss: 2303085.2500
Epoch 9/1000
92/92 [==============================] - 0s 1ms/step - loss: 2324490.7500
Epoch 10/1000
92/92 [==============================] - 0s 1ms/step - loss: 2316455.2500
Epoch 11/1000
92/92 [==============================] - 0s 1ms/step - loss: 2276504.7500
Epoch 12/1000
92/92 [====================

In [0]:
# 결과출력
print("\n Accuracy : %.4f" %(model.evaluate(x_data, y_data)[1]))

92/92 [==============================] - 0s 925us/step - loss: 3428517.0000


TypeError: ignored

### 결과 예측하기

In [11]:
data = ((23.2, 20.2, 27.3, 7.4), )
arr = np.array(data, dtype=np.float32)

xdata = arr[0:4]
xdata
print(model.predict(xdata))

[[4193.3525]]
